In [0]:
from pyspark.sql.functions import (
    col,
    when,
    current_timestamp
)

In [0]:
latest_vitals = spark.table(
    "healthcare_analytics.gold_patient_latest_vitals"
)

In [0]:
alerts = (
    latest_vitals
    .withColumn(
        "heart_rate_alert",
        when(col("heart_rate") > 120, "CRITICAL")
        .when(col("heart_rate") > 100, "HIGH")
        .otherwise("NORMAL")
    )
    .withColumn(
        "oxygen_alert",
        when(col("oxygen_level") < 90, "CRITICAL")
        .when(col("oxygen_level") < 95, "LOW")
        .otherwise("NORMAL")
    )
    .withColumn(
        "bp_alert",
        when(col("systolic_bp") > 160, "CRITICAL")
        .when(col("systolic_bp") > 140, "HIGH")
        .otherwise("NORMAL")
    )
)

In [0]:
alerts_with_severity = (
    alerts
    .withColumn(
        "overall_severity",
        when(
            (col("heart_rate_alert") == "CRITICAL") |
            (col("oxygen_alert") == "CRITICAL") |
            (col("bp_alert") == "CRITICAL"),
            "CRITICAL"
        )
        .when(
            (col("heart_rate_alert") == "HIGH") |
            (col("bp_alert") == "HIGH"),
            "HIGH"
        )
        .otherwise("NORMAL")
    )
)

In [0]:
final_alerts = (
    alerts_with_severity
    .withColumn("alert_generated_ts", current_timestamp())
    .select(
        "patient_id",
        "event_time",
        "heart_rate",
        "oxygen_level",
        "systolic_bp",
        "heart_rate_alert",
        "oxygen_alert",
        "bp_alert",
        "overall_severity",
        "alert_generated_ts",
        "source_system"
    )
)

In [0]:
(
    final_alerts.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("healthcare_analytics.gold_patient_alerts")
)

In [0]:
spark.table(
    "healthcare_analytics.gold_patient_alerts"
).display()

patient_id,event_time,heart_rate,oxygen_level,systolic_bp,heart_rate_alert,oxygen_alert,bp_alert,overall_severity,alert_generated_ts,source_system
496,2026-01-18T07:03:45.353Z,91,90,90,NORMAL,LOW,NORMAL,NORMAL,2026-01-18T22:42:37.735Z,simulated_vitals
148,2026-01-18T07:03:25.729Z,94,100,102,NORMAL,NORMAL,NORMAL,NORMAL,2026-01-18T22:42:37.735Z,simulated_vitals
463,2026-01-18T07:04:51.608Z,96,87,106,NORMAL,CRITICAL,NORMAL,CRITICAL,2026-01-18T22:42:37.735Z,simulated_vitals
392,2026-01-18T07:04:05.804Z,82,93,104,NORMAL,LOW,NORMAL,NORMAL,2026-01-18T22:42:37.735Z,simulated_vitals
243,2026-01-18T07:03:55.631Z,113,90,103,HIGH,LOW,NORMAL,HIGH,2026-01-18T22:42:37.735Z,simulated_vitals
31,2026-01-18T07:04:21.081Z,124,98,102,CRITICAL,NORMAL,NORMAL,CRITICAL,2026-01-18T22:42:37.735Z,simulated_vitals
451,2026-01-18T07:04:51.608Z,65,95,104,NORMAL,NORMAL,NORMAL,NORMAL,2026-01-18T22:42:37.735Z,simulated_vitals
251,2026-01-18T07:04:46.528Z,95,99,172,NORMAL,NORMAL,CRITICAL,CRITICAL,2026-01-18T22:42:37.735Z,simulated_vitals
458,2026-01-18T07:03:45.353Z,126,93,126,CRITICAL,LOW,NORMAL,CRITICAL,2026-01-18T22:42:37.735Z,simulated_vitals
65,2026-01-18T07:04:56.701Z,106,91,169,HIGH,LOW,CRITICAL,CRITICAL,2026-01-18T22:42:37.735Z,simulated_vitals
